In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import zipfile 
zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip').extractall('/kaggle/data')
zipfile.ZipFile('/kaggle/input/dogs-vs-cats/test1.zip').extractall('/kaggle/data')

In [ ]:
# !ls /kaggle/data/train 
image_names = os.listdir('/kaggle/data/train')
print(len(image_names))

In [ ]:
dog_list = [name for name in image_names if name[0:3] == 'dog']
cat_list = [name for name in image_names if name[0:3] == 'cat']
print(dog_list[0:5])
print(cat_list[0:5])
print(len(dog_list))
print(len(cat_list))

In [ ]:
import random
random.seed(123)

random.shuffle(dog_list)
random.shuffle(cat_list)

train_list = dog_list[0:11000] + cat_list[0:11000]
valid_list = dog_list[11000:] + cat_list[11000:]

random.shuffle(train_list)
random.shuffle(valid_list)

In [ ]:
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision.io import read_image

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
size = 128

transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((size,size)),
#     torchvision.transforms.RandomCrop((size,size))
])

class DogsAndCatsDataset(Dataset):
    def __init__(self, name_list, img_dir="/kaggle/data/train", transform=transform, target_transform=None):
        self.name_list = name_list
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.name_list)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.name_list[idx])
        image = read_image(img_path)
        if self.name_list[idx][0:3] == 'dog':
            label = 0
        elif self.name_list[idx][0:3] == 'cat':
            label = 1
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
from torch.utils.data import DataLoader

batch_size = 128

train_set = DogsAndCatsDataset(train_list)
valid_set = DogsAndCatsDataset(valid_list)

train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_set, batch_size=batch_size, shuffle=True)


In [ ]:
# train_features, train_labels = next(iter(train_dataloader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# image = train_features[0].squeeze()
# displayImage = image.permute(1,2,0)
# label = train_labels[0]
# plt.imshow(displayImage, interpolation='nearest')
# plt.show()
# print(f"Label: {label}")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv5 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv6 = nn.Conv2d(128, 128, 3, padding=1)
        self.conv7 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv8 = nn.Conv2d(256, 256, 3, padding=1)
        self.dropout2d = nn.Dropout2d(0.25)
        self.dropout = nn.Dropout(0.6)
        self.batchnorm_fc1 = nn.BatchNorm1d(256)
        self.batchnorm_fc2 = nn.BatchNorm1d(128)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.batchnorm4 = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(256 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = F.relu(x)
#         x = self.dropout2d(x)
        x = self.conv2(x)
        x = self.batchnorm1(x)
        x = F.relu(x)
        x = self.dropout2d(x)
        
        x = self.pool(x)
        
        x = self.conv3(x)
        x = self.batchnorm2(x)
        x = F.relu(x)
#         x = self.dropout2d(x)
        x = self.conv4(x)
        x = self.batchnorm2(x)
        x = F.relu(x)
        x = self.dropout2d(x)
        
        x = self.pool(x)

        x = self.conv5(x)
        x = self.batchnorm3(x)
        x = F.relu(x)
#         x = self.dropout2d(x)
        x = self.conv6(x)
        x = self.batchnorm3(x)
        x = F.relu(x)
        x = self.dropout2d(x)
        
        x = self.pool(x)
        
        x = self.conv7(x)
        x = self.batchnorm4(x)
        x = F.relu(x)
#         x = self.dropout2d(x)
        x = self.conv8(x)
        x = self.batchnorm4(x)
        x = F.relu(x)
        x = self.dropout2d(x)
        
        x = self.pool(x)

        x = torch.flatten(x, 1)
        
        x = self.fc1(x)
        x = self.batchnorm_fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.batchnorm_fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

net = Net().to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-2, momentum=0.9, weight_decay=5e-2)
# optimizer = optim.Adam(net.parameters(), lr=5e-4, weight_decay=1e-2)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=0, verbose=True)


In [ ]:
def validate():
    correct = 0
    total = 0
    running_loss = 0
    with torch.no_grad():
        for data in valid_dataloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images.float())
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            running_loss += loss.item() * batch_size
        valid_loss = running_loss / len(valid_list)
        valid_acc = 100 * correct / total

    print(f'valid loss: {valid_loss:.4f}, valid accuracy: {valid_acc:.2f}')
    return (valid_loss, valid_acc)

In [ ]:
import time 
start_time = time.time()
epochs = 50

print('Training Started')
train_losses = []
valid_losses = []
train_correct = 0
train_total = 0

for epoch in range(epochs):
    
    running_loss = 0.0
    
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = net(inputs.float())

        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch_size
        
    train_losses.append(running_loss / len(train_list))
    print(f'[{epoch + 1} epoch] loss: {running_loss / len(train_list) :.4f}, accuracy: {100 * train_correct / train_total :.2f}')
    valid_loss, valid_accu = validate()
    valid_losses.append(valid_loss)
    scheduler.step(running_loss)
    
end_time = time.time()
print(f'Finished Training. {end_time - start_time:.5f} sec')


In [ ]:
plt.plot(range(1,epochs+1), train_losses)
plt.plot(range(1,epochs+1), valid_losses)

In [ ]:
validate()

In [ ]:
# PATH = '/kaggle/model/baseline.pth'
# torch.save(net.state_dict(), PATH)